# **[PyTorch](https://pytorch.org/) Basic**

Tensorflow vs PyTorch
- Tensorflow 1.X -> low level
- Tensorflow 2.X -> high level --> with keras

When optimizing large and complex functions, such as neural networks, especially with substantial datasets, we rely on optimization techniques. PyTorch offers key functionalities to streamline this process, including:
- Automatic Differentiation: PyTorch's ([TORCH.AUTOGRAD](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#a-gentle-introduction-to-torch-autograd)) module automatically computes gradients for complex mathematical functions, making backpropagation and optimization much easier to implement. 
- Comprehensive Training Tools: PyTorch provides a wide range of utilities for training neural networks, including predefined optimizers, loss functions, and easy-to-use model building frameworks, all designed to enhance efficiency and flexibility in the training process.

[옵션] ```pip install ipywidgets``` (Jupyter notebook으로 실행하는 경우 불필요한 경고가 뜨는 것을 방지할 수 있음)

### **Tensor**

[1] Creating tensors from lists - tensors, like numpy arrays, are also multidimensional arrays, but with additional functionalities for deep learning.

You can determine the location of a tensor (whether on CPU or GPU) by using the ```tensor.device``` command.

[2] Numpy and Tensor

To improve training speed, it is recommended to minimize unnecessary copying of data in PyTorch whenever possible.

In [ ]:
import torch
import numpy as np

data = [[1, 2],[3, 4]]
x_np = np.array(data)

x_data = torch.tensor(x_np, dtype=torch.int16) 
print(x_data.shape, x_data.dtype)
print(x_data)

x_data = torch.from_numpy(x_np) 
print(x_data.shape, x_data.dtype)
print(x_data)

x_data = torch.as_tensor(x_np, dtype=torch.float) 
print(x_data.shape, x_data.dtype)
print(x_data)


ones, zeros, randon numbers

In [ ]:
shape = (2,3,)
ones_tensor = torch.ones(shape) 
zeros_tensor = torch.zeros(shape) 
rand_tensor = torch.rand(shape)

# float32

print(ones_tensor)
print(zeros_tensor)
print(rand_tensor)

In [ ]:
data = [[1, 2],[3, 4]]
x_np = np.array(data)
x_data = torch.tensor(x_np, dtype=torch.float)

x_ones = torch.ones_like(x_data) 
x_zeros = torch.zeros_like(x_data)
x_rand = torch.rand_like(x_data)

print(x_ones)
print(x_zeros)
print(x_rand)

Basic operation

In [ ]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])

c = a * b 

print(c)
print(c.sum()) 
print(c.sum().item()) 
print(type(c.sum().item())) # <class 'int'>

Slicing 



In [ ]:
import torch

a = torch.tensor([1, 2, 3, 4, 5, 6])

b1 = a[0:3]
b2 = a[3:6]

print(b1)
print(b2)

torch.cat()

In [ ]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
b = torch.tensor([[7, 8, 9], [10, 11, 12]])

print("a shape:", a.shape)
print("b shape:", b.shape)

print("-----------------------------")

t = torch.cat([a, b], dim = 0) 

print("t shape:", t.shape)
print(t)

print("-----------------------------")

t = torch.cat([a, b], dim = 1) 

print("t shape:", t.shape)
print(t)

When we want to use GPU..


In [ ]:
tensor = torch.rand(3,4)

if torch.cuda.is_available():
    tensor = tensor.to("cuda")

### Autograd

$y = 2.0 \cdot x^5 + 1.0$]

When $x=0.5$, the gradient of $y$ with respect to $x$ is:

In [ ]:
import torch
import numpy as np

x_data = torch.tensor([0.5], requires_grad=True) # requires_grad = True

y = 2.0 * x_data**5 + 1.0 

y.backward() # 미분!

x_data.grad.item()

$\frac{dy}{dx} = 10.0 \cdot x ^ 4$


##### [Exercise] Second-order derivative of a curve

$y = x^2$

Let's calculate the gradients of $y$ for each $x$ value of [-1.0, -0.75 -0.5, 0.0, 0.5, 0.75, 1.0].

In [ ]:
import torch
import numpy as np

x_samples = [-1.0, -0.75, -0.5, 0.0, 0.5, 0.75, 1.0]

grad_y_list = []

# your code

print(grad_y_list)


Let's observe the meaning of gradients.

In [1]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.xlabel("x")
plt.ylabel("y")
plt.plot(np.linspace(-1.2, 1.2, 100), np.linspace(-1.2, 1.2, 100)**2)

plt.scatter(x_samples, [x**2 for x in x_samples], c ="red")

dx = 0.1
for x, grad_y in zip(x_samples, grad_y_list): 
    if grad_y > 0.0:
        plt.plot([x, x + dx], [x**2, x**2 + grad_y * dx], color = 'blue')
    elif grad_y < 0.0:
        plt.plot([x, x - dx], [x**2, x**2 - grad_y * dx], color = 'blue')
    else:
        pass 

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import torch
import numpy as np


# x = 1.0
x = torch.tensor([1.0], requires_grad=True)

learning_rate = 1e-4


def my_func(x):
    return x**2  # y = x*x


def gradient_descent():
    global x

    y = my_func(x)

    x.grad = None  # gradient update x
    y.backward()

    with torch.no_grad():
        x -= learning_rate * x.grad


# animation code
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

fig = plt.figure(figsize=(14, 8))

axis = plt.axes(xlim=(-1.4, 1.4), ylim=(-0.2, 1.6))

(line,) = plt.plot(np.linspace(-1.2, 1.2, 100), my_func(np.linspace(-1.2, 1.2, 100)))
scatter = axis.scatter(x.detach().numpy(), my_func(x).detach().numpy(), c="red")


def init():
    scatter.set_offsets(torch.cat([x, my_func(x)], dim=0).detach().numpy())
    return (line, scatter)


def animate(i):
    gradient_descent()
    scatter.set_offsets(torch.cat([x, my_func(x)], dim=0).detach().numpy())
    return (line, scatter)


anim = FuncAnimation(fig, animate, init_func=init, frames=200, interval=1, blit=True)

plt.show()
